## 3. Report Generation

The final phase of SDMT involves aggregating evidence, validating the metrics reflected by the evidence we collected, and displaying this information in a report.

#### Preliminaries

In [1]:
import os
from pathlib import Path

# The path at which media is stored
REPORTS_DIR = Path(os.getcwd()) / "reports"
os.makedirs(REPORTS_DIR, exist_ok=True)

#### Initialize MLTE Context

MLTE contains a global context that manages the currently active _session_. Initializing the context tells MLTE how to store all of the artifacts that it produces.

In [2]:
from mlte.session import set_context, set_store

store_path = os.path.join(os.getcwd(), "store")

set_context("IrisClassifier", "0.0.1")
set_store(f"local://{store_path}")

### Validate Values and get an updated `ValidatedSpec` with `Result`s

Now that we have our `Spec` ready and we have enough evidence, we create a `SpecValidator` with our spec, and add all the `Value`s we have. With that we can validate our spec and generate an output `ValidatedSpec`, with the validation results.

In [3]:
from mlte.spec.spec import Spec
from mlte.validation.spec_validator import SpecValidator
from mlte.value.artifact import Value

# Load the specification
spec = Spec.load()

# Add all values to the validator.
spec_validator = SpecValidator(spec)
spec_validator.add_values(Value.load_all())

In [4]:
# Validate requirements and get validated details.
validated_spec = spec_validator.validate()

# ValidatedSpec also supports persistence
validated_spec.save(force=True)

#### Generate a Report

The final step of SDMT involves the generation of a report to communicate the results of model evaluation.

In [5]:
from mlte.model.shared import (
    ProblemType,
    GoalDescriptor,
    MetricDescriptor,
    ModelProductionDescriptor,
    ModelInterfaceDescriptor,
    ModelIODescriptor,
    ModelResourcesDescriptor,
    RiskDescriptor,
    DataDescriptor,
    DataClassification,
    FieldDescriptor,
    LabelDescriptor,
)
from mlte.report.artifact import (
    Report,
    SummaryDescriptor,
    PerformanceDesciptor,
    IntendedUseDescriptor,
    CommentDescriptor,
    QuantitiveAnalysisDescriptor,
)

report = Report(
    summary=SummaryDescriptor(
        problem_type=ProblemType.CLASSIFICATION, task="Iris classification"
    ),
    performance=PerformanceDesciptor(
        goals=[
            GoalDescriptor(
                description="The model should perform well.",
                metrics=[
                    MetricDescriptor(
                        description="accuracy",
                        baseline="Better than random chance.",
                    )
                ],
            )
        ],
        validated_spec_id=validated_spec.identifier,
    ),
    intended_use=IntendedUseDescriptor(
        usage_context="A handheld flower identification device.",
        production_requirements=ModelProductionDescriptor(
            deployment_platform="local server",
            capability_deployment_mechanism="API",
            interface=ModelInterfaceDescriptor(
                input=ModelIODescriptor(
                    name="i1", description="description", type="Vector[150]"
                ),
                output=ModelIODescriptor(
                    name="o1", description="description", type="Vector[3]"
                ),
            ),
            resources=ModelResourcesDescriptor(
                cpu="1", gpu="0", memory="6MiB", storage="2KiB"
            ),
        ),
    ),
    risks=RiskDescriptor(
        fp="The wrong type of flower is identified.",
        fn="The flower is not identified.",
        other="N/A",
    ),
    data=[
        DataDescriptor(
            description="Iris flower dataset.",
            classification=DataClassification.UNCLASSIFIED,
            access="None",
            labeling_method="by hand",
            fields=[
                FieldDescriptor(
                    name="Sepal length",
                    description="The length of the sepal.",
                    type="float",
                    expected_values="N/A",
                    missing_values="N/A",
                    special_values="N/A",
                )
            ],
            labels=[
                LabelDescriptor(description="Setosa", percentage=30.0),
                LabelDescriptor(description="Versicolour", percentage=30.0),
                LabelDescriptor(description="Virginica", percentage=40.0),
            ],
            policies="N/A",
            rights="N/A",
            source="https://archive.ics.uci.edu/dataset/53/iris",
        )
    ],
    comments=[
        CommentDescriptor(
            content="This model should not be used for nefarious purposes."
        )
    ],
    quantitative_analysis=QuantitiveAnalysisDescriptor(
        content="Insert graph here."
    ),
)

report.save(force=True, parents=True)